In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("Data/phisingData.csv")
data = data.astype(float)
data.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,...,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0
1,1.0,1.0,1.0,1.0,1.0,-1.0,0.0,1.0,-1.0,1.0,...,1.0,1.0,-1.0,-1.0,0.0,-1.0,1.0,1.0,1.0,-1.0
2,1.0,0.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,...,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,0.0,-1.0,-1.0
3,1.0,0.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,...,1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0
4,1.0,0.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,1.0,-1.0,-1.0,0.0,-1.0,1.0,1.0,1.0,1.0


In [3]:
print(data.isnull().sum())

having_IP_Address              0
URL_Length                     0
Shortining_Service             0
having_At_Symbol               0
double_slash_redirecting       0
Prefix_Suffix                  0
having_Sub_Domain              0
SSLfinal_State                 0
Domain_registeration_length    0
Favicon                        0
port                           0
HTTPS_token                    0
Request_URL                    0
URL_of_Anchor                  0
Links_in_tags                  0
SFH                            0
Submitting_to_email            0
Abnormal_URL                   0
Redirect                       0
on_mouseover                   0
RightClick                     0
popUpWidnow                    0
Iframe                         0
age_of_domain                  0
DNSRecord                      0
web_traffic                    0
Page_Rank                      0
Google_Index                   0
Links_pointing_to_page         0
Statistical_report             0
Result    

In [4]:
data.describe()

X = data.drop(columns=["Result"])
y = data["Result"]

In [5]:
print(y.dtypes)

float64


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=420)

In [7]:
# randomforest , decsiontree , Logistic regression

In [8]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from mlflow.models import infer_signature



In [9]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Spam-Website-Data-part2")

2025/05/26 19:05:45 INFO mlflow.tracking.fluent: Experiment with name 'Spam-Website-Data-part2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/803720941544646985', creation_time=1748307945512, experiment_id='803720941544646985', last_update_time=1748307945512, lifecycle_stage='active', name='Spam-Website-Data-part2', tags={}>

In [10]:
param_grids = {
    'RandomForest': {'n_estimators': [50, 100], 'max_depth': [5, 10]},
    'DecisionTree': {'max_depth': [5, 10], 'criterion': ['gini', 'entropy']},
    'LogisticRegression': {'C': [0.1, 1.0], 'solver': ['liblinear']}
}

models = {
    'RandomForest': RandomForestClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000)
}

for model_name , model in models.items():
    for params in ParameterGrid(param_grids[model_name]):
        with mlflow.start_run(run_name=f"{model_name}_experiment"):

            #Set params
            model.set_params(**params)
            model.fit(X_train,y_train)

            # Prediction and the metrics
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average="weighted")
            recall = recall_score(y_test, y_pred, average="weighted")

            # print the parameters
            print(f"{model_name} the Accuracy {accuracy} , Precision {precision}, Recall {recall}")

            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)

            ## Infer the model signature
            signature = infer_signature(X_train,y_train)

            model_info = mlflow.sklearn.log_model(
            sk_model = model,
            artifact_path = "NS_model",
            signature = signature,
            input_example = X_train,
            )











RandomForest the Accuracy 0.9294500723589001 , Precision 0.9294766081901537, Recall 0.9294500723589001
🏃 View run RandomForest_experiment at: http://127.0.0.1:5000/#/experiments/803720941544646985/runs/6bf468b560c74d1bbcaf85b0a06ec6c3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/803720941544646985
RandomForest the Accuracy 0.9352387843704776 , Precision 0.9356285184253387, Recall 0.9352387843704776
🏃 View run RandomForest_experiment at: http://127.0.0.1:5000/#/experiments/803720941544646985/runs/baa8983f02d244d3bd7bb8f8976689db
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/803720941544646985
RandomForest the Accuracy 0.9609261939218524 , Precision 0.9610932050510045, Recall 0.9609261939218524
🏃 View run RandomForest_experiment at: http://127.0.0.1:5000/#/experiments/803720941544646985/runs/e2d9e00ce43445a5b6399bb1194cf611
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/803720941544646985
RandomForest the Accuracy 0.9605643994211288 , Precision 0.96071

In [12]:
import mlflow
model_uri = 'runs:/e2d9e00ce43445a5b6399bb1194cf611/NS_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri)

y_pred = loaded_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)

0.9609261939218524
